In [1]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold,StratifiedGroupKFold
import numpy as np
import random
import pandas as pd
import os
from __future__ import annotations
from datasets import Dataset
from transformers import AutoTokenizer, LlamaForSequenceClassification, MistralForSequenceClassification
from transformers import DataCollatorWithPadding
from sklearn.metrics import log_loss
from transformers import TrainingArguments, Trainer
from shutil import rmtree
from scipy.special import softmax
import gc
from pathlib import Path

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TRANSFORMERS_CACHE'] = '///mnt/c/Personal/Competitions/HFCache/'
os.environ['HF_HOME'] = '///mnt/c/Personal/Competitions/HFCache/'

In [3]:
class Config:
    # General settings
    EXP_NAME = 'nb006'
    competition_name = 'h2O_llm'
    seed = 2022 #42
    debug = False
    train = True
    n_fold = 5
    TARGET_MODEL = 'HuggingFaceH4/zephyr-7b-beta' #"mistralai/Mistral-7B-Instruct-v0.1" #"mistralai/Mistral-7B-v0.1"
    DEBUG = False
    max_len = 2048
    
CFG = Config()

In [4]:
def seed_everything(seed, use_cuda = True):
    np.random.seed(seed) # cpu vars
    torch.manual_seed(seed) # cpu  vars
    random.seed(seed) # Python
    os.environ['PYTHONHASHSEED'] = str(seed) # Python hash building
    if use_cuda:
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_everything(CFG.seed)

In [5]:
data_dir = "///mnt/c/Personal/Competitions/Kaggle/h2oai-predict-the-llm/"
OUTPUT_DIR = f'///mnt/c/Personal/Competitions/Kaggle/h2oai-predict-the-llm/runs/' + CFG.EXP_NAME + "/"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

train = pd.read_csv(data_dir + "train.csv").rename(columns={'target': 'label'})
test = pd.read_csv(data_dir + "test.csv")
sample_submission = pd.read_csv(data_dir + "sample_submission.csv")

In [6]:
train.head()

,Question,Response,label
0,Explain the concept of coevolution.,Coevolution is a biological process that occur...,3
1,Is it possible that recurring fever and chills...,"Yes, recurring fever and chills can be a sympt...",4
2,Evaluate the expression 3!,The expression 3! represents the factorial of ...,1
3,What are the roles of different types of RNA i...,1. Messenger RNA (mRNA): mRNA carries genetic ...,3
4,What is the role of gene flow in population ge...,Gene flow refers to the movement of individual...,3


In [ ]:
# train['Question'] = train['Question'].str.replace('\n', '')
# train['Response'] = train['Response'].str.replace('\n', '')
# test['Question'] = test['Question'].str.replace('\n', '')
# test['Response'] = test['Response'].str.replace('\n', '')


In [7]:
gkf = StratifiedGroupKFold(n_splits=CFG.n_fold,shuffle=True,random_state=CFG.seed)

for i, (_, val_index) in enumerate(gkf.split(train, train["label"],groups=train['Question'])):
    train.loc[val_index, "fold"] = i

train = train.fillna("NA")
test = test.fillna("NA")

In [8]:
train['all_text'] = 'Question: ' + train.Question + '; Answer: ' + train.Response
test['all_text'] = 'Question: ' + test.Question + '; Answer: ' + test.Response

In [9]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    target_modules=[
        "q_proj",
        "v_proj"
    ],
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = softmax(predictions,axis=1)
    print(predictions)
    logloss_val = log_loss(labels, predictions)
    
    return {
        "logloss": logloss_val,
    }

In [11]:
def run(fld):
    
    tokenizer = AutoTokenizer.from_pretrained(CFG.TARGET_MODEL, use_fast=False)
    tokenizer.pad_token = tokenizer.eos_token
    # LlamaForSequenceClassification(
    base_model = MistralForSequenceClassification.from_pretrained(
        CFG.TARGET_MODEL,
        num_labels=7,
        cache_dir='///mnt/c/Personal/Competitions/HFCache/',
        quantization_config=bnb_config,
        device_map={"":0})
    
    # base_model.config.pretraining_tp = 1 # 1 is 7b
    base_model.config.pad_token_id = tokenizer.pad_token_id
    model = get_peft_model(base_model, peft_config)
    model.print_trainable_parameters()

    train_df = train[train['fold']!=fld].reset_index(drop=True)
    valid_df = train[train['fold']==fld].reset_index(drop=True)

    print('train shape:',train_df.shape, 'valid shape:',valid_df.shape)
    # from pandas
    train_ds = Dataset.from_pandas(train_df)
    valid_ds = Dataset.from_pandas(valid_df)

    def preprocess_function(examples, max_length=CFG.max_len):
        return tokenizer(examples["all_text"], 
                         truncation=True, 
                         max_length=max_length, 
                         padding=True)

    train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
    valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")    
    model_fold_dir = os.path.join(OUTPUT_DIR,str(fld)) 
    
    training_args = TrainingArguments(
        output_dir=model_fold_dir,
        learning_rate=3e-4,#5e-5,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=32,
        max_grad_norm=0.3,
        optim='paged_adamw_32bit',
        lr_scheduler_type="cosine",
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        push_to_hub=False,
        warmup_steps=100,
        eval_steps=50,
        save_steps = 50,
        logging_steps=50,
        report_to='none' # if DEBUG else 'wandb',
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized_ds,
        eval_dataset=valid_tokenized_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    
    trainer.train()

    trainer.save_model(output_dir=str(model_fold_dir))
    
    for path in Path(training_args.output_dir).glob("checkpoint-*"):
        if path.is_dir():
            rmtree(path)
            
    del trainer, model, base_model

    for i in range(5):
        torch.cuda.empty_cache()
        gc.collect()

In [12]:
for f in range(5):
    run(f)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at HuggingFaceH4/zephyr-7b-beta and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,873,088 || all params: 7,117,533,184 || trainable%: 0.09656559122829936
train shape: (3178, 5) valid shape: (798, 5)


Map:   0%|          | 0/3178 [00:00<?, ? examples/s]

Map:   0%|          | 0/798 [00:00<?, ? examples/s]

...

/home/rajneesh/miniconda3/envs/llm/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


[[4.2038e-03 2.2054e-06 9.7803e-01 ... 1.7639e-02 0.0000e+00 0.0000e+00]
 [6.8018e-01 3.9935e-06 1.0638e-01 ... 2.1167e-01 7.0751e-05 1.4648e-03]
 [8.0322e-02 1.4305e-05 7.6172e-01 ... 1.5784e-01 6.5565e-07 7.1526e-05]
 ...
 [1.1993e-01 2.6345e-04 1.8372e-01 ... 6.8750e-01 5.9605e-08 6.9580e-03]
 [8.8215e-04 9.9805e-01 1.8346e-04 ... 9.5367e-07 0.0000e+00 7.4005e-04]
 [3.4571e-06 6.5029e-05 5.3644e-07 ... 0.0000e+00 0.0000e+00 1.0000e+00]]


/home/rajneesh/miniconda3/envs/llm/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


[[2.646e-05 0.000e+00 1.000e+00 ... 9.763e-05 0.000e+00 0.000e+00]
 [9.316e-01 0.000e+00 1.019e-02 ... 5.817e-02 3.576e-07 4.172e-07]
 [1.634e-02 0.000e+00 9.282e-01 ... 5.533e-02 0.000e+00 0.000e+00]
 ...
 [5.127e-02 1.669e-06 2.791e-01 ... 6.699e-01 0.000e+00 1.192e-07]
 [7.153e-06 1.000e+00 9.537e-07 ... 0.000e+00 0.000e+00 2.503e-06]
 [0.000e+00 3.022e-05 0.000e+00 ... 0.000e+00 0.000e+00 1.000e+00]]


/home/rajneesh/miniconda3/envs/llm/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


In [13]:
gc.collect()

0

In [14]:
torch.cuda.empty_cache()

In [15]:
def preprocess_function(examples, max_length=CFG.max_len):
    return tokenizer(examples["all_text"], 
                     truncation=True, 
                     max_length=max_length, 
                     padding=True)
tokenizer = AutoTokenizer.from_pretrained(CFG.TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

test_ds = Dataset.from_pandas(test)
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [16]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [17]:
base_model = MistralForSequenceClassification.from_pretrained(
    CFG.TARGET_MODEL,
    num_labels=7,
    cache_dir='///mnt/c/Personal/Competitions/HFCache/',
    quantization_config=bnb_config,
    device_map={"":0})

base_model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at HuggingFaceH4/zephyr-7b-beta and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
import scipy
out = np.zeros((1001, 7))
for f in [0,2,3,4]:#(range(5)): fold 1 was really bad
    print(f'----------- Fold: {f} ----------')
    model = PeftModel.from_pretrained(base_model, f'///mnt/c/Personal/Competitions/Kaggle/h2oai-predict-the-llm/runs/nb006/{f}/')

    trainer = Trainer(model=model,
                      tokenizer=tokenizer,
                      data_collator=data_collator)
    pred_output = trainer.predict(test_tokenized_ds)
    logits = pred_output.predictions
    probits = scipy.special.softmax(logits,1)

    out += probits/4
    del trainer, model
    torch.cuda.empty_cache()    
    gc.collect()

----------- Fold: 0 ----------


----------- Fold: 2 ----------


----------- Fold: 3 ----------


----------- Fold: 4 ----------


In [19]:
out

array([[2.42165565e-01, 9.00506973e-03, 6.56509399e-02, ...,
        5.36315918e-01, 1.26384676e-01, 1.90911293e-02],
       [1.37627125e-04, 4.68301773e-03, 5.76376915e-05, ...,
        2.79188156e-04, 9.94506836e-01, 4.95254993e-04],
       [1.19209290e-07, 5.96046448e-07, 5.96046448e-08, ...,
        4.76837158e-07, 1.00000000e+00, 5.96046448e-08],
       ...,
       [4.92553711e-02, 1.31215096e-01, 3.27293396e-01, ...,
        2.95013428e-01, 1.61668360e-01, 3.06643844e-02],
       [3.54125977e-01, 1.47848129e-02, 3.08013916e-01, ...,
        3.07586670e-01, 2.63869762e-04, 1.32680535e-02],
       [2.23785639e-03, 9.34197903e-02, 1.60312653e-03, ...,
        1.09428167e-03, 5.06639481e-05, 9.01489258e-01]])

In [20]:
sample_submission.iloc[:,1:] = out

In [21]:
OUTPUT_DIR

'///mnt/c/Personal/Competitions/Kaggle/h2oai-predict-the-llm/runs/nb006/'

In [22]:
sample_submission.to_csv(f'{OUTPUT_DIR}/submit.csv',index=False)

### Smaller max len inference

In [30]:
def preprocess_function(examples, max_length=1536):
    return tokenizer(examples["all_text"], 
                     truncation=True, 
                     max_length=max_length, 
                     padding=True)
tokenizer = AutoTokenizer.from_pretrained(CFG.TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

test_ds = Dataset.from_pandas(test)
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [31]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [32]:
base_model = MistralForSequenceClassification.from_pretrained(
    CFG.TARGET_MODEL,
    num_labels=7,
    cache_dir='///mnt/c/Personal/Competitions/HFCache/',
    quantization_config=bnb_config,
    device_map={"":0})

base_model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at HuggingFaceH4/zephyr-7b-beta and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
import scipy
out = np.zeros((1001, 7))
for f in [0,2,3,4]:#(range(5)): fold 1 was really bad
    print(f'----------- Fold: {f} ----------')
    model = PeftModel.from_pretrained(base_model, f'///mnt/c/Personal/Competitions/Kaggle/h2oai-predict-the-llm/runs/nb006/{f}/')

    trainer = Trainer(model=model,
                      tokenizer=tokenizer,
                      data_collator=data_collator)
    pred_output = trainer.predict(test_tokenized_ds)
    logits = pred_output.predictions
    probits = scipy.special.softmax(logits,1)

    out += probits/4
    del trainer, model
    torch.cuda.empty_cache()    
    gc.collect()

----------- Fold: 0 ----------


----------- Fold: 2 ----------


----------- Fold: 3 ----------


----------- Fold: 4 ----------


In [34]:
sample_submission.iloc[:,1:] = out

In [35]:
# sample_submission.to_csv(f'{OUTPUT_DIR}/submit_maxlenX2.csv',index=False)
sample_submission.to_csv(f'{OUTPUT_DIR}/submit_maxlen_1536.csv',index=False)